In [1]:
### importation des librairies
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Dropout
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.layers import RepeatVector
from tensorflow.keras.layers import *
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Dropout, BatchNormalization, Bidirectional, Flatten, concatenate
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ReduceLROnPlateau
tf.random.set_seed(42)
np.random.seed(42)
from Data_Shaping import *
from models_check import *
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
print(tf.__version__)

np.set_printoptions(precision=2)

2.16.2


In [2]:
### importation des données en pandas
x_train_df = pd.read_csv("/Users/josealmeida/Desktop/Data Challenge/données/X_train_N1UvY30.csv")
y_train_df = pd.read_csv("/Users/josealmeida/Desktop/Data Challenge/données/y_train_or6m3Ta.csv")
x_test_df=pd.read_csv("/Users/josealmeida/Desktop/Data Challenge/données/X_test_m4HAPAP.csv")
x_train_df.head(120)


,obs_id,venue,order_id,action,side,price,bid,ask,bid_size,ask_size,trade,flux
0,0,4,0,A,A,0.30,0.0,0.01,100,1,False,100
1,0,4,1,A,B,-0.17,0.0,0.01,100,1,False,100
2,0,4,2,D,A,0.28,0.0,0.01,100,1,False,-100
3,0,4,3,A,A,0.30,0.0,0.01,100,1,False,100
4,0,4,4,D,A,0.37,0.0,0.01,100,1,False,-100
...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,4,11,A,B,-0.06,0.0,0.04,2,75,False,100
116,1,4,12,A,B,-0.01,0.0,0.04,2,75,False,100
117,1,4,13,A,A,0.06,0.0,0.04,2,75,False,4
118,1,2,14,A,B,-0.01,0.0,0.04,2,75,False,1


In [3]:
x_train_final_df=process_sequences(x_train_df)
print(x_train_final_df)

          obs_id  venue  order_id action side  price   bid   ask  bid_size  \
0              0      4         0      A    A   0.30  0.00  0.01       100   
1              0      4         1      A    B  -0.17  0.00  0.01       100   
2              0      4         2      D    A   0.28  0.00  0.01       100   
3              0      4         3      A    A   0.30  0.00  0.01       100   
4              0      4         4      D    A   0.37  0.00  0.01       100   
...          ...    ...       ...    ...  ...    ...   ...   ...       ...   
16079995  160799      4        61      D    A   1.32  0.01  0.06       735   
16079996  160799      0        70      A    A   0.06  0.01  0.06       735   
16079997  160799      4        71      A    A   1.26  0.01  0.06       735   
16079998  160799      4        72      A    A   1.26  0.01  0.06       735   
16079999  160799      4        73      A    B   0.00  0.01  0.06       635   

          ask_size  trade  flux  spread  order_imbalance  cumul

In [ ]:
def prepare_data_for_rnn(df):
    action_encoded=df['action'].map({'A': 0, 'D': 1, 'U': 2})
    venue_encoded=df['venue']
    side_encoded=df['side'].map({'A': 0, 'B': 1})
    trade_encoded=df['trade'].map({True: 1, False: 0})
    return df

In [11]:

# Définir les colonnes au niveau global
NUMERIC_COLS = ['price', 'bid', 'ask', 'bid_size', 'ask_size', 'flux',
               'spread', 'order_imbalance', 'cumul_volume', 
               'price_change', 'price_anomaly']

CATEGORICAL_COLS = ['action', 'side', 'venue', 'trade', 'order_id']

def prepare_data_for_rnn(df):
    # 1. Encodage des variables catégorielles
    label_encoders = {}
    
    for col in CATEGORICAL_COLS:
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[col])
        label_encoders[col] = le
    
    # 3. Reshape des données en séquences de 100
    n_sequences = len(df) // 100
    
    # Préparer les inputs pour les embeddings
    categorical_inputs = {
        col: df[f'{col}_encoded'].values.reshape(n_sequences, 100)
        for col in CATEGORICAL_COLS
    }
    
    # Préparer les features numériques
    numeric_features = df[NUMERIC_COLS].values.reshape(n_sequences, 100, len(NUMERIC_COLS))
    
    return numeric_features, categorical_inputs, label_encoders

def train_rnn_model(x_train_df, y_train, input_shape):
    # Préparer les données
    numeric_features, categorical_inputs, label_encoders = prepare_data_for_rnn(x_train_df)
    
    # Obtenir les dimensions pour les embeddings
    embedding_dims = {
        'order_id': len(label_encoders['order_id'].classes_),
        'action': len(label_encoders['action'].classes_),
        'side': len(label_encoders['side'].classes_),
        'venue': len(label_encoders['venue'].classes_),
        'trade': len(label_encoders['trade'].classes_)
    }
    
    # Créer et compiler le modèle
    model = complexed_gru_reshaped2(
        input_shape=(100, len(NUMERIC_COLS)),
        order_id_dim=embedding_dims['order_id'],
        action_dim=embedding_dims['action'],
        side_dim=embedding_dims['side'],
        venue_dim=embedding_dims['venue'],
        trade_dim=embedding_dims['trade']
    )
    
    # Préparer les inputs pour l'entraînement
    model_inputs = [
        numeric_features,
        categorical_inputs['order_id'],
        categorical_inputs['action'],
        categorical_inputs['side'],
        categorical_inputs['venue'],
        categorical_inputs['trade']
    ]
    
    # S'assurer que y_train a la bonne forme
    n_sequences = len(x_train_df) // 100
    y_train = y_train[:n_sequences]  # Prendre une étiquette par séquence
    y_train_cat = to_categorical(y_train, num_classes=24)
    
    # Répéter les étiquettes pour chaque timestep si nécessaire
    y_train_cat = np.repeat(y_train_cat[:, np.newaxis, :], 100, axis=1)
    
    # Entraîner le modèle
    history = model.fit(
        model_inputs,
        y_train_cat,
        epochs=50,
        batch_size=32,
        validation_split=0.2
    )
    
    return model, history, label_encoders



In [12]:
# Exemple d'utilisation
input_shape = (100, len(NUMERIC_COLS))  # Utilisation de la constante globale
# Assurez-vous de n'utiliser que la colonne eqt_code_cat
y_train = y_train_df['eqt_code_cat'].values  # Sélectionner uniquement la colonne de labels

model, history, encoders = train_rnn_model(x_train_final_df, y_train, input_shape)

Shape of inputs: (None, 100, 11)
Shape after concatenation and repeat: (None, 100, 24)
Shape of concatenated_embeddings: (None, 100, 24)
Shape after concatenation: (None, 100, 35)
Shape of inputs: (None, 100, 11)
Shape of concatenated_embeddings: (None, 100, 24)
Epoch 1/50
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 797s 197ms/step - accuracy: 0.1349 - loss: 2.7273 - val_accuracy: 0.2536 - val_loss: 2.2913
Epoch 2/50
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 794s 197ms/step - accuracy: 0.2458 - loss: 2.3341 - val_accuracy: 0.3580 - val_loss: 1.9737
Epoch 3/50
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 789s 196ms/step - accuracy: 0.3171 - loss: 2.1257 - val_accuracy: 0.4035 - val_loss: 1.8487
Epoch 4/50
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 784s 195ms/step - accuracy: 0.3596 - loss: 1.9954 - val_accuracy: 0.4371 - val_loss: 1.7321
Epoch 5/50
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 790s 197ms/step - accuracy: 0.3851 - loss: 1.9185 - val_accuracy: 0.4391 - val_loss: 1.7193
Epoch 6/50
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 787s 196ms/step - accurac

In [6]:
print("Valeurs uniques dans y_train:", np.unique(y_train_df))
print("\nMin:", y_train_df.min())
print("Max:", y_train_df.max())

Valeurs uniques dans y_train: [     0      1      2 ... 160797 160798 160799]

Min: obs_id          0
eqt_code_cat    0
dtype: int64
Max: obs_id          160799
eqt_code_cat        23
dtype: int64
